该notebook中的模型，可以做到与biogeme的结果基本一致，包括对于模型的Structural equation，Measurement equation以及Utility function中的参数的估计，对应研究进展20181023。

# Data and Package Prepare

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

In [2]:
train_data = pd.read_csv('optif.csv', low_memory=False)
#train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
# S=tf.Session()

In [3]:
trainable = True

In [4]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['CostPT_HWH', 'CostPT_Other']]
train_CARcom = train_data[['CostCar_HWH', 'CostCar_Other']]

train_PTasc = train_data[['TimePT', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar']]
train_SMasc = train_data[['distance_km']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

# Latent Variable Submodel

In [5]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [6]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [7]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [8]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [26]:
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
hidden = keras.layers.Dense(4, name = 'hidden', activation = 'relu')(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(hidden)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [27]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 0s 115us/step


102836.640625

In [28]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=8000, batch_size = train_latent.shape[0])

Epoch 1/8000
1545/1545 [==============================] - 0s 206us/step - loss: 102836.6406
Epoch 2/8000
1545/1545 [==============================] - 0s 4us/step - loss: 98311.4375
Epoch 3/8000
1545/1545 [==============================] - 0s 4us/step - loss: 93376.4062
Epoch 4/8000
1545/1545 [==============================] - 0s 10us/step - loss: 88473.5156
Epoch 5/8000
1545/1545 [==============================] - 0s 6us/step - loss: 83740.8984
Epoch 6/8000
1545/1545 [==============================] - 0s 4us/step - loss: 79249.2812
Epoch 7/8000
1545/1545 [==============================] - 0s 6us/step - loss: 75057.3438
Epoch 8/8000
1545/1545 [==============================] - 0s 7us/step - loss: 71172.7500
Epoch 9/8000
1545/1545 [==============================] - 0s 4us/step - loss: 67619.6641
Epoch 10/8000
1545/1545 [==============================] - 0s 5us/step - loss: 64388.6172
Epoch 11/8000
1545/1545 [==============================] - 0s 5us/step - loss: 61449.3008
Epoch 12/8000
1

1545/1545 [==============================] - 0s 6us/step - loss: 19928.6055
Epoch 93/8000
1545/1545 [==============================] - 0s 5us/step - loss: 19888.3359
Epoch 94/8000
1545/1545 [==============================] - 0s 5us/step - loss: 19849.1855
Epoch 95/8000
1545/1545 [==============================] - 0s 8us/step - loss: 19811.1172
Epoch 96/8000
1545/1545 [==============================] - 0s 6us/step - loss: 19774.0723
Epoch 97/8000
1545/1545 [==============================] - 0s 5us/step - loss: 19738.0117
Epoch 98/8000
1545/1545 [==============================] - 0s 10us/step - loss: 19702.9219
Epoch 99/8000
1545/1545 [==============================] - 0s 4us/step - loss: 19668.7734
Epoch 100/8000
1545/1545 [==============================] - 0s 4us/step - loss: 19635.5176
Epoch 101/8000
1545/1545 [==============================] - 0s 7us/step - loss: 19603.0918
Epoch 102/8000
1545/1545 [==============================] - 0s 4us/step - loss: 19571.4785
Epoch 103/8000
1545/

Epoch 183/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18387.7617
Epoch 184/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18381.7305
Epoch 185/8000
1545/1545 [==============================] - 0s 16us/step - loss: 18375.7969
Epoch 186/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18369.9668
Epoch 187/8000
1545/1545 [==============================] - 0s 6us/step - loss: 18364.2383
Epoch 188/8000
1545/1545 [==============================] - 0s 12us/step - loss: 18358.6094
Epoch 189/8000
1545/1545 [==============================] - 0s 6us/step - loss: 18353.0781
Epoch 190/8000
1545/1545 [==============================] - 0s 6us/step - loss: 18347.6445
Epoch 191/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18342.2988
Epoch 192/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18337.0469
Epoch 193/8000
1545/1545 [==============================] - 0s 6us/step - loss: 18331.88

Epoch 273/8000
1545/1545 [==============================] - 0s 15us/step - loss: 18097.6660
Epoch 274/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18096.0059
Epoch 275/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18094.3613
Epoch 276/8000
1545/1545 [==============================] - 0s 9us/step - loss: 18092.7305
Epoch 277/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18091.1172
Epoch 278/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18089.5176
Epoch 279/8000
1545/1545 [==============================] - 0s 6us/step - loss: 18087.9336
Epoch 280/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18086.3633
Epoch 281/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18084.8066
Epoch 282/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18083.2637
Epoch 283/8000
1545/1545 [==============================] - 0s 5us/step - loss: 18081.734

Epoch 363/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17988.8066
Epoch 364/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17987.9023
Epoch 365/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17987.0039
Epoch 366/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17986.1113
Epoch 367/8000
1545/1545 [==============================] - 0s 9us/step - loss: 17985.2188
Epoch 368/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17984.3340
Epoch 369/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17983.4531
Epoch 370/8000
1545/1545 [==============================] - 0s 15us/step - loss: 17982.5762
Epoch 371/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17981.7051
Epoch 372/8000
1545/1545 [==============================] - 0s 11us/step - loss: 17980.8359
Epoch 373/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17979.97

...

Epoch 7624/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.9531
Epoch 7625/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.0215
Epoch 7626/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17419.0898
Epoch 7627/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17419.1602
Epoch 7628/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.3359
Epoch 7629/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17419.3867
Epoch 7630/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.4492
Epoch 7631/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17419.4570
Epoch 7632/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.3594
Epoch 7633/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.1719
Epoch 7634/8000
1545/1545 [==============================] - 0s 5us/step - loss:

1545/1545 [==============================] - 0s 7us/step - loss: 17418.7109
Epoch 7714/8000
1545/1545 [==============================] - 0s 7us/step - loss: 17418.7637
Epoch 7715/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.8164
Epoch 7716/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.8516
Epoch 7717/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.8418
Epoch 7718/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.8086
Epoch 7719/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7676
Epoch 7720/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7227
Epoch 7721/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.8164
Epoch 7722/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.8008
Epoch 7723/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7285
Epoc

Epoch 7803/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7363
Epoch 7804/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7148
Epoch 7805/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.6836
Epoch 7806/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.8301
Epoch 7807/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7871
Epoch 7808/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7109
Epoch 7809/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.7500
Epoch 7810/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7734
Epoch 7811/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7734
Epoch 7812/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.7559
Epoch 7813/8000
1545/1545 [==============================] - 0s 5us/step - loss:

Epoch 7893/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7305
Epoch 7894/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7227
Epoch 7895/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.7383
Epoch 7896/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7422
Epoch 7897/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.7617
Epoch 7898/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.7695
Epoch 7899/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.7734
Epoch 7900/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17418.8164
Epoch 7901/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.8477
Epoch 7902/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17418.8691
Epoch 7903/8000
1545/1545 [==============================] - 0s 6us/step - loss:

Epoch 7983/8000
1545/1545 [==============================] - 0s 8us/step - loss: 17419.1230
Epoch 7984/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17419.1758
Epoch 7985/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17419.3594
Epoch 7986/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17419.5098
Epoch 7987/8000
1545/1545 [==============================] - 0s 9us/step - loss: 17419.5723
Epoch 7988/8000
1545/1545 [==============================] - 0s 6us/step - loss: 17419.5742
Epoch 7989/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17419.4492
Epoch 7990/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.1953
Epoch 7991/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17419.1016
Epoch 7992/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17418.8789
Epoch 7993/8000
1545/1545 [==============================] - 0s 5us/step - loss:

In [29]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
hidden 1
car_sys 2
indicator_ref 3
indicator_other 4
concatenate_1 5
input_y_ture 6
subtract_1 7
ll_1 8


In [30]:
idmodel.layers[8].get_weights()

[array([1.3587012, 1.1742724, 1.2000048, 1.1321788, 1.1824138, 1.1473099,
        1.3010086], dtype=float32)]

# Discrete Choice Submodel

计算效用中公共系数部分

V0 alternative specific part

In [16]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.72),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.37),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.0275),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [17]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=4.66),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-5.83),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        # cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        # return tf.add(tf.add(cl,timeCAR), self.asc_CAR)
        return tf.add(timeCAR, self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [18]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.22),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.62),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        self.beta_cl_SM = self.add_weight(name='beta_cl_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=2.),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        cl_SM = tf.multiply(inputs[0], self.beta_cl_SM)
        dist = tf.multiply(inputs[1], self.beta_dist)
        
        return tf.add(tf.add(cl_SM, dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [19]:
class TransToRandShape(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(TransToRandShape, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(TransToRandShape, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.zeros((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(TransToRandShape, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [20]:
class Add_Random(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Add_Random, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(Add_Random, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.random_normal((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(Add_Random, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [31]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
idmodel.layers[2].trainable = False
hidden = idmodel.layers[1](input_latent)
car_sys = idmodel.layers[2](hidden)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1505,random_num)), name = 'carlover')(car_sys)
carlover = Add_Random(1,name = 'carlover')(car_sys)

### common coefficients part in utility
utilcommon = keras.layers.Dense(1, name = 'util_common', use_bias=False)
PTcom = TransToRandShape(1)(utilcommon(input_PTcom))
CARcom = TransToRandShape(1)(utilcommon(input_CARcom))


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)([carlover, input_SMasc])
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [32]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 0s 163us/step


[14.498381614685059, 0.05889967456459999]

In [ ]:
# tf.keras.backend.get_value(util_V1as)
# callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=4000, batch_size = train_latent.shape[0])

Epoch 1/4000
1545/1545 [==============================] - 0s 314us/step - loss: 14.4976 - acc: 0.0595
Epoch 2/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.4906 - acc: 0.0589
Epoch 3/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4862 - acc: 0.0583
Epoch 4/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4814 - acc: 0.0602
Epoch 5/4000
1545/1545 [==============================] - 0s 31us/step - loss: 14.4776 - acc: 0.0595
Epoch 6/4000
1545/1545 [==============================] - 0s 39us/step - loss: 14.4709 - acc: 0.0602
Epoch 7/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4664 - acc: 0.0621
Epoch 8/4000
1545/1545 [==============================] - 0s 37us/step - loss: 14.4592 - acc: 0.0615
Epoch 9/4000
1545/1545 [==============================] - 0s 37us/step - loss: 14.4551 - acc: 0.0615
Epoch 10/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4415 - 

Epoch 82/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.0899 - acc: 0.2867
Epoch 83/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.0670 - acc: 0.2919
Epoch 84/4000
1545/1545 [==============================] - 0s 29us/step - loss: 10.0504 - acc: 0.2939
Epoch 85/4000
1545/1545 [==============================] - 0s 29us/step - loss: 10.0369 - acc: 0.2958
Epoch 86/4000
1545/1545 [==============================] - 0s 31us/step - loss: 10.0233 - acc: 0.2990
Epoch 87/4000
1545/1545 [==============================] - 0s 29us/step - loss: 10.0089 - acc: 0.3016
Epoch 88/4000
1545/1545 [==============================] - 0s 29us/step - loss: 9.9946 - acc: 0.3016
Epoch 89/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.9953 - acc: 0.3049
Epoch 90/4000
1545/1545 [==============================] - 0s 34us/step - loss: 9.9905 - acc: 0.3049
Epoch 91/4000
1545/1545 [==============================] - 0s 31us/step - loss: 9.989

Epoch 163/4000
1545/1545 [==============================] - 0s 29us/step - loss: 8.3772 - acc: 0.3366
Epoch 164/4000
1545/1545 [==============================] - 0s 30us/step - loss: 8.3072 - acc: 0.3379
Epoch 165/4000
1545/1545 [==============================] - 0s 28us/step - loss: 8.2480 - acc: 0.3398
Epoch 166/4000
1545/1545 [==============================] - 0s 30us/step - loss: 8.1680 - acc: 0.3411
Epoch 167/4000
1545/1545 [==============================] - 0s 31us/step - loss: 8.0753 - acc: 0.3411
Epoch 168/4000
1545/1545 [==============================] - 0s 30us/step - loss: 7.9963 - acc: 0.3417
Epoch 169/4000
1545/1545 [==============================] - 0s 30us/step - loss: 7.8812 - acc: 0.3443
Epoch 170/4000
1545/1545 [==============================] - 0s 30us/step - loss: 7.7725 - acc: 0.3495
Epoch 171/4000
1545/1545 [==============================] - 0s 30us/step - loss: 7.6412 - acc: 0.3540
Epoch 172/4000
1545/1545 [==============================] - 0s 29us/step - loss: 7

Epoch 244/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0986 - acc: 0.6977
Epoch 245/4000
1545/1545 [==============================] - 0s 31us/step - loss: 3.0823 - acc: 0.6990
Epoch 246/4000
1545/1545 [==============================] - 0s 28us/step - loss: 3.0818 - acc: 0.6990
Epoch 247/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0754 - acc: 0.6990
Epoch 248/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0578 - acc: 0.6997
Epoch 249/4000
1545/1545 [==============================] - 0s 29us/step - loss: 3.0470 - acc: 0.7010
Epoch 250/4000
1545/1545 [==============================] - 0s 32us/step - loss: 3.0410 - acc: 0.7029
Epoch 251/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0342 - acc: 0.7029
Epoch 252/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0151 - acc: 0.7029
Epoch 253/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3

Epoch 325/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.5695 - acc: 0.7256
Epoch 326/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.5587 - acc: 0.7269
Epoch 327/4000
1545/1545 [==============================] - 0s 29us/step - loss: 2.5475 - acc: 0.7256
Epoch 328/4000
1545/1545 [==============================] - 0s 31us/step - loss: 2.5453 - acc: 0.7262
Epoch 329/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.5498 - acc: 0.7275
Epoch 330/4000
1545/1545 [==============================] - 0s 29us/step - loss: 2.5342 - acc: 0.7275
Epoch 331/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.5297 - acc: 0.7288
Epoch 332/4000
1545/1545 [==============================] - 0s 29us/step - loss: 2.5260 - acc: 0.7282
Epoch 333/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.5250 - acc: 0.7294
Epoch 334/4000
1545/1545 [==============================] - 0s 29us/step - loss: 2

...

Epoch 3293/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6159 - acc: 0.7288
Epoch 3294/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6165 - acc: 0.7346
Epoch 3295/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6189 - acc: 0.7359
Epoch 3296/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6179 - acc: 0.7353
Epoch 3297/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6149 - acc: 0.7366
Epoch 3298/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6162 - acc: 0.7346
Epoch 3299/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6164 - acc: 0.7366
Epoch 3300/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6175 - acc: 0.7327
Epoch 3301/4000
1545/1545 [==============================] - 0s 28us/step - loss: 0.6192 - acc: 0.7359
Epoch 3302/4000
1545/1545 [==============================] - 0s 30us/step

Epoch 3373/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6158 - acc: 0.7359
Epoch 3374/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6173 - acc: 0.7366
Epoch 3375/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6147 - acc: 0.7346
Epoch 3376/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6171 - acc: 0.7353
Epoch 3377/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6168 - acc: 0.7366
Epoch 3378/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6141 - acc: 0.7333
Epoch 3379/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6144 - acc: 0.7340
Epoch 3380/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6183 - acc: 0.7372
Epoch 3381/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6150 - acc: 0.7359
Epoch 3382/4000
1545/1545 [==============================] - 0s 29us/step

Epoch 3453/4000
1545/1545 [==============================] - 0s 40us/step - loss: 0.6168 - acc: 0.7366
Epoch 3454/4000
1545/1545 [==============================] - 0s 38us/step - loss: 0.6159 - acc: 0.7353
Epoch 3455/4000
1545/1545 [==============================] - 0s 36us/step - loss: 0.6153 - acc: 0.7366
Epoch 3456/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6167 - acc: 0.7353
Epoch 3457/4000
1545/1545 [==============================] - 0s 38us/step - loss: 0.6164 - acc: 0.7333
Epoch 3458/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6180 - acc: 0.7359
Epoch 3459/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6179 - acc: 0.7327
Epoch 3460/4000
1545/1545 [==============================] - 0s 35us/step - loss: 0.6154 - acc: 0.7333
Epoch 3461/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6162 - acc: 0.7353
Epoch 3462/4000
1545/1545 [==============================] - 0s 35us/step

Epoch 3533/4000
1545/1545 [==============================] - 0s 57us/step - loss: 0.6148 - acc: 0.7327
Epoch 3534/4000
1545/1545 [==============================] - 0s 34us/step - loss: 0.6164 - acc: 0.7346
Epoch 3535/4000
1545/1545 [==============================] - 0s 35us/step - loss: 0.6178 - acc: 0.7385
Epoch 3536/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6159 - acc: 0.7346
Epoch 3537/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6185 - acc: 0.7314
Epoch 3538/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6151 - acc: 0.7282
Epoch 3539/4000
1545/1545 [==============================] - 0s 35us/step - loss: 0.6177 - acc: 0.7353
Epoch 3540/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6141 - acc: 0.7366
Epoch 3541/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6163 - acc: 0.7307
Epoch 3542/4000
1545/1545 [==============================] - 0s 29us/step

Epoch 3613/4000
1545/1545 [==============================] - 0s 35us/step - loss: 0.6164 - acc: 0.7301
Epoch 3614/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6174 - acc: 0.7314
Epoch 3615/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6158 - acc: 0.7333
Epoch 3616/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6149 - acc: 0.7359
Epoch 3617/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6153 - acc: 0.7379
Epoch 3618/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6155 - acc: 0.7392
Epoch 3619/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6165 - acc: 0.7327
Epoch 3620/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6175 - acc: 0.7301
Epoch 3621/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6153 - acc: 0.7359
Epoch 3622/4000
1545/1545 [==============================] - 0s 33us/step

In [25]:
lossList=[]
for i in range(100):
    loss_value=model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0],verbose=0)[0]
    lossList.append(loss_value)
print(np.mean(lossList))

0.6116320580244065


In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [41]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
carlover 2
input_PTasc 3
input_PTcom 4
input_CARasc 5
input_CARcom 6
input_SMasc 7
v0_as_2 8
util_common 9
v1_as_2 10
v2_as_2 11
V0 12
V1 13
V2 14
prediction 15


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [55]:
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, model.layers[output_index[i]].get_weights()))

car_sys         [array([[-0.12619334],
       [-0.09635082],
       [ 0.36885747],
       [-0.3496434 ],
       [ 0.6265547 ],
       [-0.06594128],
       [ 0.05845913],
       [ 0.6226478 ],
       [ 0.21824066],
       [ 0.09964096],
       [-0.04249339],
       [ 0.2796297 ],
       [-0.6184652 ]], dtype=float32), array([2.5612953], dtype=float32)]
carlover        []
v0_as_4         [array([[-2.0230172]], dtype=float32), array([[1.5660286]], dtype=float32), array([[-0.02796068]], dtype=float32)]
util_common     [array([[-1.172912]], dtype=float32), array([[-0.47131765]], dtype=float32)]
v1_as_4         [array([[5.2523074]], dtype=float32), array([[-6.434374]], dtype=float32)]
v2_as_4         [array([[-1.5794731]], dtype=float32), array([[-0.4133285]], dtype=float32), array([[1.9687463]], dtype=float32)]
V0              []
V1              []
V2              []
prediction      []


In [47]:
model.layers[9].get_weights()

[array([[-1.392449]], dtype=float32), array([[-0.55060613]], dtype=float32)]